# `mammos_entity.io`: reading and writing entities

In [1]:
from pathlib import Path

import mammos_entity as me
import pandas as pd
from mammos_entity import units as u

## Supported file format

`mammos_entity` defines a yaml file format to store entities (or other compatible entity-likes) or an entity collection. The file structure is similar to an `EntityCollection`:
- the file has a description (and some other metadata such as the used ontology; [TODO this is not implemented yet])
- the file can contain any number of entity-likes; it stores data and all associated metadata (such as a link to the ontology and units)

`mammos_entity.io` can read and write these yaml files. All details of the file format are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html).

## Writing individual entities
We create some artificial data that we can write to file:

In [2]:
Ms = me.Ms([600, 650, 700], "kA/m", description="Evaluated using UppASD with 70000 Monte Carlo steps.")
T = me.T([1, 2, 3])
theta_angle = [0, 0.5, 0.7] * u.rad
demag_factor = me.Entity("DemagnetizingFactor", [1 / 3, 1 / 3, 1 / 3])
comments = ["Some comment", "Some other comment", "A third comment"]

We can write entity-likes to a yaml file (extension .yaml or .yml) as shown in the following cell. Names of the keyword arguments determine keys under `data` in the file:

In [3]:
me.io.entities_to_file(
    "example.yaml",
    description="Example yaml file.\nThe description can have multiple lines.\n\nLines can also be empty.",
    Ms=Ms,
    T=T,
    angle=theta_angle,
    demag_factor=demag_factor,
    comment=comments,
)

This has produced the following file:

In [4]:
print(Path("example.yaml").read_text())

metadata:
  version: v2
  description: |-
    Example yaml file.
    The description can have multiple lines.

    Lines can also be empty.
data:
  Ms:
    ontology_label: SpontaneousMagnetization
    description: Evaluated using UppASD with 70000 Monte Carlo steps.
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: kA / m
    value: [600.0, 650.0, 700.0]
  T:
    ontology_label: ThermodynamicTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [1.0, 2.0, 3.0]
  angle:
    ontology_label: null
    description: ''
    ontology_iri: null
    unit: rad
    value: [0.0, 0.5, 0.7]
  demag_factor:
    ontology_label: DemagnetizingFactor
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e
    unit: ''
    value: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

## Reading
We can read it back in and get an `EntityCollection`:

In [5]:
content = me.io.entities_from_file("example.yaml")
content

EntityCollection(
    description='Example yaml file.\nThe description can have multiple lines.\n\nLines can also be empty.',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m', description='Evaluated using UppASD with 70000 Monte Carlo steps.'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=['Some comment', 'Some other comment', 'A third comment'],
)

We can access elements as explained in the [EntityCollection tutorial](entity-collection.ipynb):

In [6]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K')

## Writing an `EntityCollection`

Entity collections provide a method `to_file` to write the collection to a yaml file:

In [7]:
col = me.EntityCollection(T=me.T([10, 20, 30]), Tc=me.Tc(300), Ms=me.Ms([[1, 2], [3, 4]], "MA/m"))
col.to_file("example2.yml")

On disk, we have the following file. This example also shows that we can have data of arbitrary shape.

In [8]:
print(Path("example2.yml").read_text())

metadata:
  version: v2
  description: ''
data:
  T:
    ontology_label: ThermodynamicTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [10.0, 20.0, 30.0]
  Tc:
    ontology_label: CurieTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_6b5af5a8_a2d8_4353_a1d6_54c9f778343d
    unit: K
    value: 300.0
  Ms:
    ontology_label: SpontaneousMagnetization
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: MA / m
    value: [[1.0, 2.0], [3.0, 4.0]]



We read it back in as before and get a new `EntityCollection`:

In [9]:
content = me.io.entities_from_file("example2.yml")
content

EntityCollection(
    description='',
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([10., 20., 30.]), unit='K'),
    Tc=Entity(ontology_label='CurieTemperature', value=np.float64(300.0), unit='K'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([[1., 2.],
           [3., 4.]]), unit='MA / m'),
)

We can acces individual elements of the returned object as before.

In [10]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([10., 20., 30.]), unit='K')

### Load, modify and save an `EntityCollection`

A common use case is loading an entity file, appending/modifying elements and saving it to a new file. We have just loaded a file and have the variable `content` containing its content as EntityCollection. We will now change the content and save it to a new file.

Manipulating an `Entity` directly is not possible because entities are immutable. Entities can however be overwritten or added to an existing collection:

In [11]:
# manipulate entities in collection
content.Ms = me.Ms(content.Ms.q.to("kA/m"))  # convert to different unit
content.A = me.A([6e-12, 7e-12, 8e-12])  # new additional entity

# update description
content.description = "Modified version of `example2.yaml`.\nMs has been converted to kA/m; additional entity A;"

content

EntityCollection(
    description='Modified version of `example2.yaml`.\nMs has been converted to kA/m; additional entity A;',
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([10., 20., 30.]), unit='K'),
    Tc=Entity(ontology_label='CurieTemperature', value=np.float64(300.0), unit='K'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([[1000., 2000.],
           [3000., 4000.]]), unit='kA / m'),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([6.e-12, 7.e-12, 8.e-12]), unit='J / m'),
)

We can write the `EntityCollection` back to disk:

In [12]:
content.to_file("example2_modified.yaml")

On disk we now have the following file:

In [13]:
print(Path("example2_modified.yaml").read_text())

metadata:
  version: v2
  description: |-
    Modified version of `example2.yaml`.
    Ms has been converted to kA/m; additional entity A;
data:
  T:
    ontology_label: ThermodynamicTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [10.0, 20.0, 30.0]
  Tc:
    ontology_label: CurieTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_6b5af5a8_a2d8_4353_a1d6_54c9f778343d
    unit: K
    value: 300.0
  Ms:
    ontology_label: SpontaneousMagnetization
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: kA / m
    value: [[1000.0, 2000.0], [3000.0, 4000.0]]
  A:
    ontology_label: ExchangeStiffnessConstant
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_526ed2a5-a017-590e-8eb8-8a900f2b3b78
    unit: J / m
    value: [6.0e-12, 7.0e-12, 8.0e-12]



## Writing a CSV file

Sometimes it can be necessary to get the numerical data as CSV file to further process it in some other software. We can get such a file by converting an `EntityCollection` to a `pandas.DataFrame` and subsequently write that to a CSV file. Both, the dataframe and the CSV file format deal with tabular data, so this only works if all entities in our collection are one-dimensional and have the same length, i.e. our collection can be represented as a table.

We create a new collection re-using the entities defined near the top of this notebook:

In [14]:
collection = me.EntityCollection(Ms=Ms, T=T, theta_angle=theta_angle, demag_factor=demag_factor, comments=comments)

We can first convert the collection to a table, here including units in the header column:

In [15]:
df = collection.to_dataframe(include_units=True)
df

,Ms (kA / m),T (K),theta_angle (rad),demag_factor,comments
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


We can now write this to a file. In this example we exclude the index (the first column without a heading) from the csv file. The `to_csv` method has many additional options to control details of the file, refer to the pandas documentation for more information.

In [16]:
df.to_csv("example.csv", index=False)

The file on disk has the following content:

In [17]:
print(Path("example.csv").read_text())

Ms (kA / m),T (K),theta_angle (rad),demag_factor,comments
600.0,1.0,0.0,0.3333333333333333,Some comment
650.0,2.0,0.5,0.3333333333333333,Some other comment
700.0,3.0,0.7,0.3333333333333333,A third comment



## Converting unformatted files and tables to `mammos yaml`

Users may wish to update other files to the `mammos yaml` format in order to make use of the additional functionality. 
Details of the file format are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html).

Converting your “raw” data into this format involves three main steps:

1. Load your file into python (e.g. with `pandas`).

2. Create an `Entity`, `quanity`, or similar out of each column (by assigning the correct ontology term and/or units).

3. Export the result with `entities_to_file`.


First let's create a file so we can see an example of how to do the conversion. We will create the following structure:
```dat
1 10.0 1.6083568305976572 -16778187.088808443
1 9.0 1.6083393931987826 -15498304.121589921
...
```

This file is quite basic, in particular, there are no headers, no units and no ontology information. The file uses a space as separator rather than a comm.

Only the user knows what each of the columns are.
In this example first column is the configuration type, the second column is the value of $\mu_0 H_{\mathsf{ext}}$ in Tesla, the third column is the magnetic polarisation in Tesla and the last column is the energy density in J/m$^3$.

In [18]:
Path("example.dat").write_text("""\
1 10.0 1.6083568305976572 -16778187.088808443
1 9.0 1.6083393931987826 -15498304.121589921
1 8.0 1.6083184361075116 -14218436.37373519
1 7.0 1.608292941666901 -12938587.029585946
1 6.0 1.6082614950059932 -11658760.230932372
""");

We can use `pandas` to read the file into python:

In [19]:
df = pd.read_csv("example.dat", sep=" ", names=["configuration_type", "mu0_Hext", "J", "energy_density"])
df

,configuration_type,mu0_Hext,J,energy_density
0,1,10.0,1.608357,-1.677819e+07
1,1,9.0,1.608339,-1.549830e+07
2,1,8.0,1.608318,-1.421844e+07
3,1,7.0,1.608293,-1.293859e+07
4,1,6.0,1.608261,-1.165876e+07


To rewrite this in a `mammos` format, we then need to associate each column with an entity, quantity, or another python object. Now is also time to do any data manipulation (such as changing units).

In this example we:
- convert configuration type to a `numpy` array.
- convert magnetic flux density ($\mu_0 H_{\mathsf{ext}}$) to the external magnetic field Entity using `mammos_units` for the relevant conversions.
- convert magnetic polarisation to the corresponding entity.
- convert energy density to the corresponding entity.

We add all entities directly to a new `EntityCollection`:

In [20]:
data = me.EntityCollection(
    configuration_type=df["configuration_type"].to_numpy(),
    H=me.H((df["mu0_Hext"].to_numpy() * u.T).to("A/m", equivalencies=u.magnetic_flux_field())),
    J=me.J(df["J"], "T"),
    energy_density=me.Entity("EnergyDensity", df["energy_density"], u.J / u.m**3),
)

We can now write the `mammos yaml` file:

In [21]:
data.to_file("example.yaml")

Looking at the file produced we can see the data is now in the correct format with the ontology information included:

In [22]:
print(Path("example.yaml").read_text())

metadata:
  version: v2
  description: ''
data:
  configuration_type:
    ontology_label: null
    description: ''
    ontology_iri: null
    unit: null
    value: [1, 1, 1, 1, 1]
  H:
    ontology_label: ExternalMagneticField
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_da08f0d3-fe19-58bc-8fb6-ecc8992d5eb3
    unit: A / m
    value: [7957747.150262763, 7161972.435236487, 6366197.72021021, 5570423.005183934,
      4774648.290157658]
  J:
    ontology_label: MagneticPolarisation
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_74a096dd_cc83_4c7e_b704_0541620ff18d
    unit: T
    value: [1.6083568305976572, 1.6083393931987826, 1.6083184361075116, 1.608292941666901,
      1.6082614950059932]
  energy_density:
    ontology_label: EnergyDensity
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_56258d3a-f2ee-554e-af99-499dd8620457
    unit: J / m3
    value: [-16778187.088808443, -15498304.12